In [42]:
import pandas as pd
import numpy as np
import folium
from folium import Choropleth
from folium.plugins import HeatMap
import datetime

ModuleNotFoundError: No module named 'folium'

## Loading EarthQuakes and Tectonic Plates Dataset

In [ ]:
earthquake_data_frame = pd.read_csv('../datasets/earth_quakes_dataset.csv')
tectonic_plate_data_frame = pd.read_csv('../datasets/tectonic_plates_dataset.csv')
earthquake_data_frame.head()


In [ ]:
tectonic_plate_data_frame.head(100000)

## Cleaning EarthQuakes Dataset

In [ ]:
not_needed_columns_to_drop = ['Depth Error', 'Depth Seismic Stations', 'Magnitude Type', 'Magnitude Error', 'Magnitude Seismic Stations', 'Azimuthal Gap', 'Horizontal Distance', 'Horizontal Error', 'Root Mean Square', 'Source', 'Location Source', 'Magnitude Source', 'Status', 'ID']
earthquake_data_frame.drop(not_needed_columns_to_drop, axis=1, inplace=True)
earthquake_data_frame.count()


## Selecting records with only earthquakes

In [ ]:
earthquake_data_frame = earthquake_data_frame[earthquake_data_frame['Type'] == "Earthquake"].reset_index(drop=True)
earthquake_data_frame.head(100000)

## Finding bad dates

In [ ]:
lengths = earthquake_data_frame["Date"].str.len()
lengths.value_counts()

## Viewing found bad dates

In [ ]:
bad_dates = np.where([lengths == 24])[1]
print("Bad dates:", bad_dates)
earthquake_data_frame.loc[bad_dates]

## Fixing identified bad dates and set date format

In [ ]:
earthquake_data_frame.loc[3359, "Date"] = "02/23/1975"
earthquake_data_frame.loc[7384, "Date"] = "04/28/1985"
earthquake_data_frame.loc[20470, "Date"] = "03/13/2011"
earthquake_data_frame['Date']= pd.to_datetime(earthquake_data_frame["Date"])
earthquake_data_frame.info()  #Check date type for changes

## Finding bad time

In [ ]:
lengths = earthquake_data_frame["Time"].str.len()
lengths.value_counts()

## Viewing found bad time

In [ ]:
bad_time = np.where([lengths == 24])[1]
print("Bad time:", bad_time)
earthquake_data_frame.loc[bad_time]

## Fixing bad time and changing time format

In [ ]:
earthquake_data_frame.loc[3359, "Time"] = "02:58:41"
earthquake_data_frame.loc[7384, "Time"] = "02:53:41"
earthquake_data_frame.loc[20470, "Time"] = "02:23:34"
earthquake_data_frame['Time']= pd.to_timedelta(earthquake_data_frame['Time'])
earthquake_data_frame.info() ##View new type

## Visualizing Tectonic Plates Boundaries

In [ ]:
tectonic = folium.Map(tiles="cartodbpositron", zoom_start=5)

plates = list(tectonic_plate_data_frame["plate"].unique())
for plate in plates:
    plate_vals = tectonic_plate_data_frame[tectonic_plate_data_frame["plate"] == plate]
    latitudes = plate_vals["lat"].values
    longitudes = plate_vals["lon"].values
    points = list(zip(latitudes, longitudes))
    indexes = [None] + [i + 1 for i, x in enumerate(points) if i < len(points) - 1 and abs(x[1] - points[i + 1][1]) > 300] + [None]
    for i in range(len(indexes) - 1):
        folium.vector_layers.PolyLine(points[indexes[i]:indexes[i+1]], popup=plate, color="#58508d", fill=False, ).add_to(tectonic)

tectonic